In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator


In [ ]:
filename = 'GOOG.csv'  # "data/solusdt_15m_data.csv"  # 
symbol = filename
df = pd.read_csv(filename)
df.info()

datetime_name = 'Date'  # 'open_datetime'  # 
price_name = 'Close'

df[datetime_name] = pd.to_datetime(df[datetime_name])   # , unit='ms')
df.corr()
#dr = df.set_axis(df[datetime_name], copy=False)
df = df[[datetime_name, price_name]]

df.head()

In [ ]:
import plotly
import plotly.graph_objects as go
print("Plotly Version: ",plotly.__version__)


In [ ]:
trace = go.Scatter(
    x = df[datetime_name],
    y = df[price_name],
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = f'{symbol} Price',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close (Dollars)"}
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [ ]:
close_data = df[price_name].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df[datetime_name][:split]
date_test = df[datetime_name][split:]

print(len(close_train))
print(len(close_test))


In [ ]:
look_back = 15


train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 50
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


In [ ]:
prediction = model.predict_generator(test_generator)
# prediction


In [ ]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))


trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = f'{symbol} Price',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()


In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    print(num_prediction)
    last_date = df[datetime_name].values[-1]
    print(last_date)
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1,  freq='15T').tolist()
    print(prediction_dates)
    return prediction_dates

num_prediction = 4
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

trace1 = go.Scatter(
    x = df[datetime_name].tolist(),
    y = close_data,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = f'{symbol} Price',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()